In [ ]:
from pyspark.sql import SparkSession

# Initialize SparkSession
spark = SparkSession \
    .builder \
    .appName("DatawarehouseSpark Application") \
    .getOrCreate()
 
inputFileHire ='/home/mose/Downloads/CSVS/client_hiring_dt.csv' #"/user/user17/mosedata_proj/input/client_hiring_dt.csv"
#inputFileBio ='/home/mose/Downloads/CSVS/client_bio_dt.csv'  #"/user/user17/mosedata_proj/input/client_bio_dt.csv"
#inputFileCom = '/home/mose/Downloads/CSVS/client_communication_dt.csv' #"/user/user17/mosedata_proj/input/client_communication_dt.csv"
#inputFileAct = '/home/mose/Downloads/CSVS/client_activities_dt.csv' #"/user/user17/mosedata_proj/input/client_activities_dt.csv"
#inputFileFact = '/home/mose/Downloads/CSVS/client_fact_ft.csv' #"/user/user17/mosedata_proj/input/client_fact_ft.csv"

 
# Create DataFrame from CSV file
dfM =  spark.read.format("csv").option("sep", ",").option("inferSchema", "true").option("header", "true").load(inputFileHire)
#dfR =  spark.read.format("csv").option("sep", ",").option("inferSchema", "true").option("header", "true").load(inputFileBio)
#dfCom =  spark.read.format("csv").option("sep", ",").option("inferSchema", "true").option("header", "true").load(inputFileCom)
#dfAct =  spark.read.format("csv").option("sep", ",").option("inferSchema", "true").option("header", "true").load(inputFileAct)
#dfFact =  spark.read.format("csv").option("sep", ",").option("inferSchema", "true").option("header", "true").load(inputFileFact)


# Print the schema of the DataFrames
dfM.printSchema()
dfR.printSchema()
dfCom.printSchema()
dfAct.printSchema()
dfFact.printSchema()


#dfR.groupBy('movieId').count().show(50)
#dfM.where("director='Martin Brest'").join(dfR,"movieId").select("movieId","director","user_name","rating").show(50)


# Stop the Spark Session
spark.stop()


In [13]:
%%writefile dataframes.py
from pyspark.sql import SparkSession

# Initialize SparkSession
spark = SparkSession \
    .builder \
    .appName("DatawarehouseSpark Application") \
    .getOrCreate()

#Input the data
inputFileHire ='/user/user17/mosedata_proj/input/client_hiring_dt.csv'
inputFileBio ='/user/user17/mosedata_proj/input/client_bio_dt.csv'
inputFileCom = '/user/user17/mosedata_proj/input/client_communication_dt.csv'
inputFileAct = '/user/user17/mosedata_proj/input/client_activities_dt.csv'
inputFileFact = '/user/user17/mosedata_proj/input/client_fact_ft.csv'

#Create data frames from sources. Tables were converted to csvs now being converted to dataframes
dfHire =  spark.read.format("csv").option("sep", ",").option("inferSchema", "true").option("header", "true").load(inputFileHire)
dfBio =  spark.read.format("csv").option("sep", ",").option("inferSchema", "true").option("header", "true").load(inputFileBio)
dfCom =  spark.read.format("csv").option("sep", ",").option("inferSchema", "true").option("header", "true").load(inputFileCom)
dfAct =  spark.read.format("csv").option("sep", ",").option("inferSchema", "true").option("header", "true").load(inputFileAct)
dfFact =  spark.read.format("csv").option("sep", ",").option("inferSchema", "true").option("header", "true").load(inputFileFact)

# Print the schema of the DataFrames
dfHire.printSchema() 
dfBio.printSchema()
dfCom.printSchema()
dfAct.printSchema()
dfFact.printSchema()

#Clients from different service branches
dfBio.groupBy('service_branch__c').count().show()

#Types of jobs the guys got hired in
dfHire.groupBy('job_function_hired_in__c').count().show()

#Temporary objects from which sql statements are run for each dataframe
dfHire.createOrReplaceTempView("dfHire_sql")
dfBio.createOrReplaceTempView("dfBio_sql")
dfCom.createOrReplaceTempView("dfCom_sql")
dfAct.createOrReplaceTempView("dfAct_sql")
dfFact.createOrReplaceTempView("dfFAct_sql")

#Mixed information from all 5 tables of the database
spark.sql(''' Select hires.hired, bio.service_rank__c , 
              bio.service_branch__c , fact.yearsinservice, fact.reg_afterservice_years,
              com.responsive__c, act.finalized_hhusa_revised_resume_on_file__c as resume_done
              from dfHire_sql hires
              inner join dfBio_sql bio on bio.id = hires.id 
              inner join dfFAct_sql fact on fact.id = hires.id
              inner join dfCom_sql com on com.id = hires.id
              inner join dfAct_sql act on act.id = hires.id
              where hires.hired = "1"
              
              ''').show()

#Account create date by month of the year
spark.sql( """SELECT count(id) , 
              Extract(Month From create_ddate) as month 
              From  dfAct_sql  
              group by month 
              ORDER BY month ASC """).show()


#Registration for services by year
spark.sql( """SELECT count(id) , 
Extract(Year From create_ddate) as year_registered 
              From  dfAct_sql  
              group by year_registered
              ORDER BY year_registered ASC """).show()

#Hired by Service branch
spark.sql(""" Select count(hires.id) as hired,bio.service_branch__c 
              From dfHire_sql hires 
              inner join dfBio_sql bio on bio.id = hires.id
              Where hires.hired = 1
              group by bio.service_branch__c
              order by hired asc
              """).show()

#Not hired by Service Branch
spark.sql(""" Select count(hires.id) as Not_hired,bio.service_branch__c 
              From dfHire_sql hires 
              inner join dfBio_sql bio on bio.id = hires.id
              Where hires.hired = 0
              group by bio.service_branch__c
              order by Not_hired asc
              """).show()

#Hired by rank
spark.sql(""" Select count(hires.id) as hired,bio.service_rank__c
              From dfHire_sql hires 
              inner join dfBio_sql bio on bio.id = hires.id
              Where hires.hired = 0
              group by bio.service_rank__c
              order by hired desc
              """).show()

#Prefered method of contact for the hired
spark.sql(""" select count(com.id) as count, com.preferred_method_of_contact__c 
              from dfCom_sql com 
              inner join dfHire_sql hires on com.id = hires.id
              where hires.hired = 1
              group by com.preferred_method_of_contact__c 
              order by count
          """).show()

#Prefered method of contact for the Unhired
spark.sql(""" select count(com.id) as count, com.preferred_method_of_contact__c 
              from dfCom_sql com 
              inner join dfHire_sql hires on com.id = hires.id
              where hires.hired = 0
              group by com.preferred_method_of_contact__c 
              order by count
          """).show()



Overwriting dataframes.py


In [1]:
from pyspark.sql import SparkSession

# Initialize SparkSession
spark = SparkSession \
    .builder \
    .appName("DatawarehouseSpark Application") \
    .getOrCreate()

In [2]:
from pyspark.ml.feature import StringIndexer, VectorIndexer
from pyspark.sql.types import StringType, IntegerType
from pyspark.sql.functions import col, regexp_replace, split, udf
from pyspark.ml.feature import Word2Vec
from pyspark.ml.feature import FeatureHasher
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import IDF
from pyspark.ml.feature import CountVectorizer
from pyspark.ml.feature import StandardScaler
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import PCA
from datetime import datetime
from pyspark.ml.clustering import KMeans
from pyspark.sql.types import *
from pyspark.ml import Pipeline
import re
import copy

In [3]:
inputFileHire ='/home/mose/Downloads/CSVS/client_hiring_dt.csv' #"/user/user17/mosedata_proj/input/client_hiring_dt.csv"
inputFileBio ='/home/mose/Downloads/CSVS/client_bio_dt.csv'  #"/user/user17/mosedata_proj/input/client_bio_dt.csv"
inputFileCom = '/home/mose/Downloads/CSVS/client_communication_dt.csv' #"/user/user17/mosedata_proj/input/client_communication_dt.csv"
inputFileAct = '/home/mose/Downloads/CSVS/client_activities_dt.csv' #"/user/user17/mosedata_proj/input/client_activities_dt.csv"
inputFileFact = '/home/mose/Downloads/CSVS/client_fact_ft.csv' #"/user/user17/mosedata_proj/input/client_fact_ft.csv"

In [4]:
# Create DataFrame from CSV file
dfHire =  spark.read.format("csv").option("sep", ",").option("inferSchema", "true").option("header", "true").load(inputFileHire)
dfBio =  spark.read.format("csv").option("sep", ",").option("inferSchema", "true").option("header", "true").load(inputFileBio)
dfCom =  spark.read.format("csv").option("sep", ",").option("inferSchema", "true").option("header", "true").load(inputFileCom)
dfAct =  spark.read.format("csv").option("sep", ",").option("inferSchema", "true").option("header", "true").load(inputFileAct)
dfFact =  spark.read.format("csv").option("sep", ",").option("inferSchema", "true").option("header", "true").load(inputFileFact)

In [5]:
# Print the schema of the DataFrames
dfHire.printSchema() 
dfBio.printSchema()
dfCom.printSchema()
dfAct.printSchema()
dfFact.printSchema()

root
 |-- id: string (nullable = true)
 |-- dsubimittedforhire: string (nullable = true)
 |-- dconfirmedforhire: string (nullable = true)
 |-- difsubmitconfirmed: integer (nullable = true)
 |-- startdate: string (nullable = true)
 |-- datdifconfirstart: integer (nullable = true)
 |-- job_type__c: string (nullable = true)
 |-- job_function_hired_in__c: string (nullable = true)
 |-- hiring_account__c: string (nullable = true)
 |-- hire_heroes_usa_confirmed_hire__c: integer (nullable = true)
 |-- hired_location__c: string (nullable = true)
 |-- hired_zip_code__c: string (nullable = true)
 |-- industry_hired_in__c: string (nullable = true)
 |-- hired_but_still_active_and_looking__c: string (nullable = true)
 |-- hired: string (nullable = true)
 |-- client_hiring_dt_id: string (nullable = true)

root
 |-- id: string (nullable = true)
 |-- client__c: integer (nullable = true)
 |-- client_type__c: string (nullable = true)
 |-- service_rank__c: string (nullable = true)
 |-- service_branch__c: 

In [6]:
#Clients from different service branches
dfBio.groupBy('service_branch__c').count().show()

+-----------------+-----+
|service_branch__c|count|
+-----------------+-----+
|          Marines|13691|
|           Spouse|    2|
|  Merchant Marine|    1|
|             null|37153|
|        Air Force|13416|
|             Army|51770|
|      Coast Guard|  837|
|   Not Applicable|    4|
|             Navy|15567|
+-----------------+-----+



In [7]:
#Types of jobs the guys got hired in
dfHire.groupBy('job_function_hired_in__c').count().show()

+------------------------+------+
|job_function_hired_in__c| count|
+------------------------+------+
|    IT - Help Desk/Su...|   287|
|    Skilled Labor/Trades|   484|
|    Purchasing/Procur...|    73|
|                   Sales|   697|
|    Firefighter/EMT/E...|    71|
|                 Science|    28|
|    Training/Instruct...|  1113|
|    Supply Chain/Logi...|   892|
|             Engineering|   465|
|    Media/Journalism/...|    54|
|         Banking/Finance|   522|
|              Healthcare|   900|
|    Quality Assurance...|   238|
|              Accounting|   171|
|    IT - Computer Sci...|    41|
|                    null|109255|
|    Business Development|   196|
|    Facilities Manage...|   165|
|              Consultant|   456|
|    Installation/Main...|  1223|
+------------------------+------+
only showing top 20 rows



In [8]:
#Temporary objects from which sql statements are run for each dataframe
dfHire.createOrReplaceTempView("dfHire_sql")
dfBio.createOrReplaceTempView("dfBio_sql")
dfCom.createOrReplaceTempView("dfCom_sql")
dfAct.createOrReplaceTempView("dfAct_sql")
dfFact.createOrReplaceTempView("dfFAct_sql")

In [9]:
#Mixed information from all 5 tables of the database
spark.sql(''' Select hires.hired, bio.service_rank__c , 
              bio.service_branch__c , fact.yearsinservice, fact.reg_afterservice_years,
              com.responsive__c, act.finalized_hhusa_revised_resume_on_file__c as resume_done
              from dfHire_sql hires
              inner join dfBio_sql bio on bio.id = hires.id 
              inner join dfFAct_sql fact on fact.id = hires.id
              inner join dfCom_sql com on com.id = hires.id
              inner join dfAct_sql act on act.id = hires.id
              where hires.hired = "1"
              
              ''').show()

+-----+---------------+-----------------+--------------+----------------------+-------------+-----------+
|hired|service_rank__c|service_branch__c|yearsinservice|reg_afterservice_years|responsive__c|resume_done|
+-----+---------------+-----------------+--------------+----------------------+-------------+-----------+
|    1|            O-3|             Army|         11.04|                 -0.33|            1|          1|
|    1|            E-9|             Army|         30.18|                 -0.15|            1|          1|
|    1|            E-4|             Army|           9.8|                 -0.73|            1|          1|
|    1|            E-4|             Army|          2.81|                  0.43|            1|          1|
|    1|            E-4|          Marines|          4.47|                  4.45|            1|          1|
|    1|            O-2|             Army|          4.38|                 -0.51|            1|          1|
|    1|            O-3|             Army|     

In [10]:
#Account create date by month of the year
spark.sql( """SELECT count(id) , 
              Extract(Month From create_ddate) as month 
              From  dfAct_sql  
              group by month 
              ORDER BY month ASC """).show()

+---------+-----+
|count(id)|month|
+---------+-----+
|    11982|    1|
|     9626|    2|
|    10152|    3|
|    12579|    4|
|    10978|    5|
|    10672|    6|
|    10815|    7|
|    11912|    8|
|    10845|    9|
|    11866|   10|
|    12022|   11|
|     8992|   12|
+---------+-----+



In [11]:
#Registration for services by year
spark.sql( """SELECT count(id) , 
Extract(Year From create_ddate) as year_registered 
              From  dfAct_sql  
              group by year_registered
              ORDER BY year_registered ASC """).show()

+---------+---------------+
|count(id)|year_registered|
+---------+---------------+
|     2825|           2007|
|     2571|           2008|
|     1769|           2009|
|     2255|           2010|
|     2732|           2011|
|     4258|           2012|
|    13128|           2013|
|    13593|           2014|
|    13248|           2015|
|    25175|           2016|
|    22933|           2017|
|    26448|           2018|
|     1506|           2019|
+---------+---------------+



In [12]:
#Hired by Service branch
spark.sql(""" Select count(hires.id) as hired,bio.service_branch__c 
              From dfHire_sql hires 
              inner join dfBio_sql bio on bio.id = hires.id
              Where hires.hired = 1
              group by bio.service_branch__c
              order by hired asc
              """).show()

+-----+-----------------+
|hired|service_branch__c|
+-----+-----------------+
|    1|   Not Applicable|
|  170|      Coast Guard|
| 1145|             null|
| 2423|          Marines|
| 2879|        Air Force|
| 3055|             Navy|
|10417|             Army|
+-----+-----------------+



In [13]:
#Not hired by Service Branch
spark.sql(""" Select count(hires.id) as Not_hired,bio.service_branch__c 
              From dfHire_sql hires 
              inner join dfBio_sql bio on bio.id = hires.id
              Where hires.hired = 0
              group by bio.service_branch__c
              order by Not_hired asc
              """).show()

+---------+-----------------+
|Not_hired|service_branch__c|
+---------+-----------------+
|        2|   Not Applicable|
|      667|      Coast Guard|
|     9382|             null|
|    10524|        Air Force|
|    11260|          Marines|
|    12500|             Navy|
|    41314|             Army|
+---------+-----------------+



In [14]:
#Hired by rank
spark.sql(""" Select count(hires.id) as hired,bio.service_rank__c
              From dfHire_sql hires 
              inner join dfBio_sql bio on bio.id = hires.id
              Where hires.hired = 1
              group by bio.service_rank__c
              order by hired desc
              """).show()

+-----+---------------+
|hired|service_rank__c|
+-----+---------------+
| 3807|            E-5|
| 3683|            E-4|
| 2795|            E-7|
| 2775|            E-6|
| 1281|            E-8|
| 1156|           null|
| 1142|            O-3|
|  648|            E-3|
|  564|            O-4|
|  510|            O-5|
|  463|            E-9|
|  294|            O-2|
|  186|            O-6|
|  178|            W-3|
|  148|            W-4|
|  137|            W-2|
|  126|            E-2|
|   79|            O-1|
|   75|            E-1|
|   27|            W-5|
+-----+---------------+
only showing top 20 rows



In [15]:
#Prefered method of contact for the hired
spark.sql(""" select count(com.id) as count, com.preferred_method_of_contact__c 
              from dfCom_sql com 
              inner join dfHire_sql hires on com.id = hires.id
              where hires.hired = 1
              group by com.preferred_method_of_contact__c 
              order by count
          """).show()

+-----+------------------------------+
|count|preferred_method_of_contact__c|
+-----+------------------------------+
|    7|                          Mail|
|   23|                      LinkedIn|
|  158|                          Text|
| 1892|                          null|
| 2626|                        E-Mail|
|15384|                     Telephone|
+-----+------------------------------+



In [16]:
#Prefered method of contact for the Unhired
spark.sql(""" select count(com.id) as count, com.preferred_method_of_contact__c 
              from dfCom_sql com 
              inner join dfHire_sql hires on com.id = hires.id
              where hires.hired = 0
              group by com.preferred_method_of_contact__c 
              order by count
          """).show()

+-----+------------------------------+
|count|preferred_method_of_contact__c|
+-----+------------------------------+
|   28|                          Mail|
|   33|                      LinkedIn|
|  378|                          Text|
|12405|                        E-Mail|
|31506|                          null|
|41299|                     Telephone|
+-----+------------------------------+



In [17]:
#Select variables that determine whether one is going to be employed or not, store those varibles in a new dataframe
dfspark= spark.sql(""" select fact.yearsinservice as S_years , fact.reg_afterservice_years as R_years,
               act.resume_tailoring_tips__c as Resume_Tips,act.finalized_hhusa_revised_resume_on_file__c as Resume_OnFile, bio.client__c as Client, bio.client_type__c as Client_Type, bio.service_rank__c as Service_R
              ,bio.service_branch__c as Service_B, hire.hire_heroes_usa_confirmed_hire__c as HHUSA_hire, com.preferred_method_of_contact__c as Com_Method, com.responsive__c
              ,com.active__c, act.created_linkedin_account__c as Created_Linkedin, bio.highest_level_of_education_completed__c as Educ,  hire.hired
              ,bio.primary_military_occupational_specialty__c as Occupation
               
              from dfFact_sql fact
              
              inner join dfAct_sql act on act.id = fact.id
              inner join dfBio_sql bio on bio.id = fact.id
              inner join dfHire_sql hire on hire.id = fact.id
              inner join dfCom_sql com on com.id = fact.id
              
              where bio.client_type__c != '' and bio.service_branch__c !='' and bio.service_rank__c !='' 
              and com.preferred_method_of_contact__c !='' and fact.yearsinservice < 53 and fact.yearsinservice > 0 
              and fact.reg_afterservice_years >- 32 and fact.reg_afterservice_years < 53 and com.active__c !=''
              and bio.highest_level_of_education_completed__c !='' and hire.hired  !='' and hire.hired !='No' and hire.hired != 3

          """)
dfspark.persist()

DataFrame[S_years: double, R_years: double, Resume_Tips: int, Resume_OnFile: int, Client: int, Client_Type: string, Service_R: string, Service_B: string, HHUSA_hire: int, Com_Method: string, responsive__c: int, active__c: string, Created_Linkedin: int, Educ: string, hired: string, Occupation: string]

In [18]:
# Get the number of clients in the description above
dfspark.count()

51093

In [19]:
# Change the data type of the the hired column from string to double
dfspark = dfspark.withColumn("hired", dfspark["hired"].cast(DoubleType()))

In [20]:
# Confirm the labels in the hired column 0 represents the client was not hired 1 represents they were hired. 
dfspark.select('hired').distinct().show()

+-----+
|hired|
+-----+
|  0.0|
|  1.0|
+-----+



In [21]:
# The data types of the data frame 
dfspark.dtypes

[('S_years', 'double'),
 ('R_years', 'double'),
 ('Resume_Tips', 'int'),
 ('Resume_OnFile', 'int'),
 ('Client', 'int'),
 ('Client_Type', 'string'),
 ('Service_R', 'string'),
 ('Service_B', 'string'),
 ('HHUSA_hire', 'int'),
 ('Com_Method', 'string'),
 ('responsive__c', 'int'),
 ('active__c', 'string'),
 ('Created_Linkedin', 'int'),
 ('Educ', 'string'),
 ('hired', 'double'),
 ('Occupation', 'string')]

In [22]:
#Hash the features through a hash function to convert the features into vectors for the dfspark dataframe,
# Convert strings to numerical data, the output column is features. The target is not included in this conversion.
funchash = FeatureHasher(inputCols= ['S_years','R_years','Resume_Tips','Resume_OnFile','Client','Client_Type','Service_R',\
 'Service_B','HHUSA_hire','Com_Method','responsive__c','active__c','Created_Linkedin','Educ','Occupation'],
                       outputCol="features")

dffeatures = funchash.transform(dfspark)
dffeatures.persist()
dffeatures.show(1)

+-------+-------+-----------+-------------+------+-----------------+---------+---------+----------+----------+-------------+---------+----------------+---------------+-----+----------+--------------------+
|S_years|R_years|Resume_Tips|Resume_OnFile|Client|      Client_Type|Service_R|Service_B|HHUSA_hire|Com_Method|responsive__c|active__c|Created_Linkedin|           Educ|hired|Occupation|            features|
+-------+-------+-----------+-------------+------+-----------------+---------+---------+----------+----------+-------------+---------+----------------+---------------+-----+----------+--------------------+
|   8.11|  -0.37|          1|            1|     1|Online Registrant|      E-5|     Army|         0| Telephone|            1|      Yes|               1|High School/GED|  0.0|       91F|(262144,[52387,65...|
+-------+-------+-----------+-------------+------+-----------------+---------+---------+----------+----------+-------------+---------+----------------+---------------+-----+---

In [28]:
dffeatures.head()

Row(S_years=8.11, R_years=-0.37, Resume_Tips=1, Resume_OnFile=1, Client=1, Client_Type='Online Registrant', Service_R='E-5', Service_B='Army', HHUSA_hire=0, Com_Method='Telephone', responsive__c=1, active__c='Yes', Created_Linkedin=1, Educ='High School/GED', hired=0.0, Occupation='91F', features=SparseVector(262144, {52387: 1.0, 65085: 8.11, 69515: 1.0, 81887: 1.0, 119601: 1.0, 132517: 1.0, 138738: 1.0, 166656: -0.37, 207405: 1.0, 220379: 1.0, 230454: 1.0, 239098: 1.0, 246347: 1.0, 249041: 1.0, 251060: 0.0}))

In [27]:
#pca = PCA(k=6, inputCol="features", outputCol="pcaFeatures")
modelpca = pca.fit(dffeatures)

Py4JJavaError: An error occurred while calling o164.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 77.0 failed 1 times, most recent failure: Lost task 0.0 in stage 77.0 (TID 2012, mose-Inspiron-5520.lan, executor driver): java.lang.IllegalArgumentException: axpy only supports adding to a dense vector but got type class org.apache.spark.mllib.linalg.SparseVector.
	at org.apache.spark.mllib.linalg.BLAS$.axpy(BLAS.scala:68)
	at org.apache.spark.mllib.feature.PCA.$anonfun$fit$3(PCA.scala:52)
	at scala.collection.Iterator$$anon$10.next(Iterator.scala:459)
	at scala.collection.Iterator$SliceIterator.next(Iterator.scala:271)
	at scala.collection.Iterator.foreach(Iterator.scala:941)
	at scala.collection.Iterator.foreach$(Iterator.scala:941)
	at scala.collection.AbstractIterator.foreach(Iterator.scala:1429)
	at scala.collection.generic.Growable.$plus$plus$eq(Growable.scala:62)
	at scala.collection.generic.Growable.$plus$plus$eq$(Growable.scala:53)
	at 

In [29]:
dffeatures.columns

['S_years',
 'R_years',
 'Resume_Tips',
 'Resume_OnFile',
 'Client',
 'Client_Type',
 'Service_R',
 'Service_B',
 'HHUSA_hire',
 'Com_Method',
 'responsive__c',
 'active__c',
 'Created_Linkedin',
 'Educ',
 'hired',
 'Occupation',
 'features']

In [30]:
#Confirm that the dataframe is active, We are going to use this data to train our machine learning algorithms.
dffeatures.select('features','hired').show()

+--------------------+-----+
|            features|hired|
+--------------------+-----+
|(262144,[52387,65...|  0.0|
|(262144,[52387,65...|  0.0|
|(262144,[65085,69...|  0.0|
|(262144,[65085,69...|  0.0|
|(262144,[65085,69...|  0.0|
|(262144,[64366,65...|  0.0|
|(262144,[54411,65...|  0.0|
|(262144,[65085,69...|  0.0|
|(262144,[65085,69...|  0.0|
|(262144,[65085,69...|  0.0|
|(262144,[11074,65...|  0.0|
|(262144,[52387,65...|  0.0|
|(262144,[40267,65...|  0.0|
|(262144,[65085,69...|  0.0|
|(262144,[52387,65...|  0.0|
|(262144,[65085,69...|  0.0|
|(262144,[48884,52...|  0.0|
|(262144,[52387,65...|  0.0|
|(262144,[65085,69...|  0.0|
|(262144,[65085,69...|  0.0|
+--------------------+-----+
only showing top 20 rows



In [31]:
#Split the data into training and testing data for the training of the algorithms and testing of the algorithms 
A,B=dffeatures.randomSplit([0.1,0.9])

In [32]:
train_data,test_data=A.randomSplit([0.65,0.35])

In [33]:
train_data.count()

3328

In [34]:
dffeatures.select('features').show()

+--------------------+
|            features|
+--------------------+
|(262144,[52387,65...|
|(262144,[52387,65...|
|(262144,[65085,69...|
|(262144,[65085,69...|
|(262144,[65085,69...|
|(262144,[64366,65...|
|(262144,[54411,65...|
|(262144,[65085,69...|
|(262144,[65085,69...|
|(262144,[65085,69...|
|(262144,[11074,65...|
|(262144,[52387,65...|
|(262144,[40267,65...|
|(262144,[65085,69...|
|(262144,[52387,65...|
|(262144,[65085,69...|
|(262144,[48884,52...|
|(262144,[52387,65...|
|(262144,[65085,69...|
|(262144,[65085,69...|
+--------------------+
only showing top 20 rows



In [35]:
#from pyspark.ml.linalg import SparseVector
from pyspark.ml.feature import Normalizer

Norm = Normalizer(inputCol="features", outputCol="features_norm", p=1)
dffeatures_norm = Norm.transform(dffeatures)

In [36]:
dffeatures_norm.select('features').show(1,False)

+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|features                                                                                                                                                                        |
+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|(262144,[52387,65085,69515,81887,119601,132517,138738,166656,207405,220379,230454,239098,246347,249041,251060],[1.0,8.11,1.0,1.0,1.0,1.0,1.0,-0.37,1.0,1.0,1.0,1.0,1.0,1.0,0.0])|
+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
only showing top 1 row



In [37]:
dffeatures_norm.columns

['S_years',
 'R_years',
 'Resume_Tips',
 'Resume_OnFile',
 'Client',
 'Client_Type',
 'Service_R',
 'Service_B',
 'HHUSA_hire',
 'Com_Method',
 'responsive__c',
 'active__c',
 'Created_Linkedin',
 'Educ',
 'hired',
 'Occupation',
 'features',
 'features_norm']

In [38]:
train_data,test_data=dffeatures_norm.randomSplit([0.7,0.3])

In [39]:
# Train the decision tree classifier, to predict whether one is hired or not, using the hired column and features. 
def treedec(label, features):
    decision_tree = DecisionTreeClassifier(labelCol=label, featuresCol=features)
    return decision_tree

In [40]:
# Now let us train the model, while timing how long it takes run this task. 

# Start time of training the model 
start = datetime.now()
def traindec_tree(train_data, algo): 
    # The model 
    model1 = algo.fit(train_data)
    return model1
# Print the duration
#traindec_tree(train_data, treedec('hired','features'))
#End of time of the model 
end = datetime.now()
print('Duration it takes to run the algorithm: {}'.format(end - start))

Duration it takes to run the algorithm: 0:00:00.000186


In [41]:
# Make predictions using the test set of the dataset ... Store the predictions in a new dataframe prediction 
predictions = traindec_tree(train_data, treedec('hired','features_norm')).transform(test_data)

In [29]:
# Check out the new dataframe predictions to view the prediction, hired and features columns 
predictions.select("prediction", "hired", "features").show()

+----------+-----+--------------------+
|prediction|hired|            features|
+----------+-----+--------------------+
|       1.0|  1.0|(262144,[52387,65...|
|       0.0|  0.0|(262144,[65085,69...|
|       0.0|  0.0|(262144,[65085,69...|
|       1.0|  0.0|(262144,[65085,69...|
|       0.0|  0.0|(262144,[52387,65...|
|       0.0|  0.0|(262144,[48884,65...|
|       1.0|  1.0|(262144,[52387,65...|
|       0.0|  0.0|(262144,[65085,69...|
|       1.0|  1.0|(262144,[65085,69...|
|       1.0|  1.0|(262144,[65085,69...|
|       1.0|  1.0|(262144,[65085,69...|
|       1.0|  1.0|(262144,[65085,69...|
|       1.0|  1.0|(262144,[65085,69...|
|       0.0|  0.0|(262144,[65085,69...|
|       0.0|  0.0|(262144,[65085,68...|
|       0.0|  0.0|(262144,[19683,65...|
|       0.0|  0.0|(262144,[48884,65...|
|       1.0|  1.0|(262144,[65085,69...|
|       0.0|  0.0|(262144,[65085,69...|
|       1.0|  1.0|(262144,[65085,69...|
+----------+-----+--------------------+
only showing top 20 rows



In [105]:
# Now lets evaluate the algorithm performance
def evaluate_algo(df,label,Algorithm_name  ):
    classification_evaluator = MulticlassClassificationEvaluator(
    labelCol=label, predictionCol="prediction")
    accuracy = classification_evaluator.evaluate(df)
    print(Algorithm_name )
    print("Test Error  = %g " % (1.0 - accuracy))
    print("Accuracy   = %g " % accuracy)

#evaluate_algo(predictions,'hired','Decision Tree')

In [31]:
# Train a RandomForest model.
from datetime import datetime

start_time = datetime.now()
# do your work here
rf = RandomForestClassifier(labelCol="hired", featuresCol="features", numTrees=17)
end_time = datetime.now()

print('Duration: {}'.format(end_time - start_time))

def randomforest(label, features, numTrees):
    
    random_forest = RandomForestClassifier(labelCol=label, featuresCol=features, numTrees=numTrees)
    
    return random_forest

Duration: 0:00:00.140405


In [32]:
from datetime import datetime

start_time = datetime.now()
# do your work here
#modelrf = rf.fit(train_data)

def random_forestfit(randf,train_data):
    """ Takes in a dataframe, and a random forest model """
    model= randf.fit(train_data)
    return model
    

end_time = datetime.now()

print('Duration: {}'.format(end_time - start_time))

Duration: 0:00:00.000133


In [33]:
# Make predictions.
predictions2 = random_forestfit(randomforest('hired', 'features', 17),train_data).transform(test_data)


In [34]:
# Select example rows to display.
predictions2.select('prediction','hired','features').show()  #.select("prediction", "indexedLabel", "features").show(5)

+----------+-----+--------------------+
|prediction|hired|            features|
+----------+-----+--------------------+
|       0.0|  1.0|(262144,[52387,65...|
|       0.0|  0.0|(262144,[65085,69...|
|       0.0|  0.0|(262144,[65085,69...|
|       0.0|  0.0|(262144,[65085,69...|
|       0.0|  0.0|(262144,[52387,65...|
|       0.0|  0.0|(262144,[48884,65...|
|       0.0|  1.0|(262144,[52387,65...|
|       0.0|  0.0|(262144,[65085,69...|
|       0.0|  1.0|(262144,[65085,69...|
|       0.0|  1.0|(262144,[65085,69...|
|       0.0|  1.0|(262144,[65085,69...|
|       0.0|  1.0|(262144,[65085,69...|
|       0.0|  1.0|(262144,[65085,69...|
|       0.0|  0.0|(262144,[65085,69...|
|       0.0|  0.0|(262144,[65085,68...|
|       0.0|  0.0|(262144,[19683,65...|
|       0.0|  0.0|(262144,[48884,65...|
|       0.0|  1.0|(262144,[65085,69...|
|       0.0|  0.0|(262144,[65085,69...|
|       0.0|  1.0|(262144,[65085,69...|
+----------+-----+--------------------+
only showing top 20 rows



In [35]:
evaluate_algo(predictions2,'hired','Random Forest')

Random Forest
Test Error  = 0.454042 
Accuracy   = 0.545958 


In [45]:
def cluster(k,df,features):
    """ Takes in k as an int, the dataframe, and features """
    kmeans = KMeans(k=k, seed=1)  # 2 clusters here
    model = kmeans.fit(df.select(features))
    return model

In [47]:
transformeddf = cluster(4,dffeatures,'features').transform(dffeatures)

In [48]:
transformeddf.select('prediction').distinct().show()

+----------+
|prediction|
+----------+
|         1|
|         3|
|         2|
|         0|
+----------+



In [61]:
transformeddf.select('Service_B','Service_R','hired','prediction').distinct().show()

+-----------+---------+-----+----------+
|  Service_B|Service_R|hired|prediction|
+-----------+---------+-----+----------+
|  Air Force|      O-4|  0.0|         1|
|       Navy|      E-1|  0.0|         3|
|  Air Force|      E-8|  1.0|         3|
|       Army|      O-4|  1.0|         1|
|Coast Guard|      O-5|  0.0|         1|
|  Air Force|      E-5|  0.0|         2|
|       Army|      O-7|  0.0|         1|
|       Army|     CW-2|  0.0|         1|
|       Navy|      O-2|  1.0|         3|
|       Army|      W-3|  1.0|         3|
|    Marines|      O-2|  1.0|         3|
|Coast Guard|      O-3|  1.0|         3|
|  Air Force|      O-6|  0.0|         1|
|  Air Force|      O-4|  1.0|         3|
|       Army|      O-1|  0.0|         1|
|       Army|      O-3|  1.0|         1|
|  Air Force|      E-3|  0.0|         0|
|       Navy|      O-4|  0.0|         2|
|  Air Force|      E-5|  1.0|         2|
|       Army|      E-9|  0.0|         1|
+-----------+---------+-----+----------+
only showing top

In [62]:
#Information about individual clusters in term of years service men stay in the army and when they register for services
transformeddf.select('S_years','R_years').describe().show()

+-------+-----------------+------------------+
|summary|          S_years|           R_years|
+-------+-----------------+------------------+
|  count|            51093|             51093|
|   mean|13.25944708668499|1.9484418609202803|
| stddev| 8.55804960106253| 6.346271266872357|
|    min|             0.01|            -29.16|
|    max|            52.33|             52.58|
+-------+-----------------+------------------+



In [63]:
#Get cluster information Cluster 0 in terms of years service men spend on service. 
cluster0 = transformeddf.filter(transformeddf.prediction==0)

In [64]:
cluster0.select('S_years','R_years').describe().show()

+-------+------------------+------------------+
|summary|           S_years|           R_years|
+-------+------------------+------------------+
|  count|              2925|              2925|
|   mean| 5.224564102564105|22.217131623931618|
| stddev|2.9455794494751912| 8.277066032403162|
|    min|              0.01|             11.33|
|    max|             21.19|             52.58|
+-------+------------------+------------------+



In [53]:
#Get cluster information Cluster 1
cluster1 = transformeddf.filter(transformeddf.prediction==1)

In [54]:
cluster1.select('S_years','R_years').describe().show()

+-------+-----------------+-------------------+
|summary|          S_years|            R_years|
+-------+-----------------+-------------------+
|  count|            18935|              18935|
|   mean|22.77949405862156|-0.6843913387906007|
| stddev|4.482330273547443| 2.2575649893207146|
|    min|             14.0|             -29.16|
|    max|            52.33|               7.92|
+-------+-----------------+-------------------+



In [55]:
#Get cluster information Cluster 2
cluster2 = transformeddf.filter(transformeddf.prediction==2)

In [56]:
cluster2.select('S_years','R_years').describe().show()

+-------+------------------+-----------------+
|summary|           S_years|          R_years|
+-------+------------------+-----------------+
|  count|              1929|             1929|
|   mean|19.301316744427176|9.179803006739244|
| stddev| 4.354761181256989|4.399028957656696|
|    min|              8.92|             2.32|
|    max|             39.37|            26.82|
+-------+------------------+-----------------+



In [57]:
#Get cluster information Cluster 3
cluster3 = transformeddf.filter(transformeddf.prediction==3)

In [58]:
cluster3.select('S_years','R_years').describe().show()

+-------+------------------+------------------+
|summary|           S_years|           R_years|
+-------+------------------+------------------+
|  count|             27304|             27304|
|   mean| 7.091309698212723|1.0920648989159096|
| stddev|3.2336080116487533|3.1435676760042863|
|    min|              0.01|            -10.07|
|    max|             15.08|             11.83|
+-------+------------------+------------------+



In [41]:
%%writefile largescale.py

from datetime import datetime
# Start time of training the model 
start = datetime.now()

from pyspark.sql.functions import col, regexp_replace, split
from pyspark.sql import SparkSession
from pyspark.ml.feature import FeatureHasher
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.clustering import KMeans
from pyspark.sql.types import *
import copy




# Initialize SparkSession
spark = SparkSession \
    .builder \
    .appName("Large Scale Computing Application ") \
    .getOrCreate()


# The dataset 
inputFileHire ='/home/mose/Downloads/CSVS/client_hiring_dt.csv' #"/user/user17/mosedata_proj/input/client_hiring_dt.csv"
inputFileBio ='/home/mose/Downloads/CSVS/client_bio_dt.csv'  #"/user/user17/mosedata_proj/input/client_bio_dt.csv"
inputFileCom = '/home/mose/Downloads/CSVS/client_communication_dt.csv' #"/user/user17/mosedata_proj/input/client_communication_dt.csv"
inputFileAct = '/home/mose/Downloads/CSVS/client_activities_dt.csv' #"/user/user17/mosedata_proj/input/client_activities_dt.csv"
inputFileFact = '/home/mose/Downloads/CSVS/client_fact_ft.csv' #"/user/user17/mosedata_proj/input/client_fact_ft.csv"

# Create DataFrames from CSV files
dfHire =  spark.read.format("csv").option("sep", ",").option("inferSchema", "true").option("header", "true").load(inputFileHire)
dfBio =  spark.read.format("csv").option("sep", ",").option("inferSchema", "true").option("header", "true").load(inputFileBio)
dfCom =  spark.read.format("csv").option("sep", ",").option("inferSchema", "true").option("header", "true").load(inputFileCom)
dfAct =  spark.read.format("csv").option("sep", ",").option("inferSchema", "true").option("header", "true").load(inputFileAct)
dfFact =  spark.read.format("csv").option("sep", ",").option("inferSchema", "true").option("header", "true").load(inputFileFact)

# Print the schemas of the DataFrames
dfHire.printSchema() 
dfBio.printSchema()
dfCom.printSchema()
dfAct.printSchema()
dfFact.printSchema()


#Clients from different service branches
dfBio.groupBy('service_branch__c').count().show()


#Types of jobs the guys got hired in
dfHire.groupBy('job_function_hired_in__c').count().show()


#Temporary objects from which sql statements can be run for each dataframe
dfHire.createOrReplaceTempView("dfHire_sql")
dfBio.createOrReplaceTempView("dfBio_sql")
dfCom.createOrReplaceTempView("dfCom_sql")
dfAct.createOrReplaceTempView("dfAct_sql")
dfFact.createOrReplaceTempView("dfFAct_sql")

#Account create date by month of the year
spark.sql( """SELECT count(id) , 
              Extract(Month From create_ddate) as month 
              From  dfAct_sql  
              group by month 
              ORDER BY month ASC """).show()

#Hired by Service branch
spark.sql(""" Select count(hires.id) as hired,bio.service_branch__c 
              From dfHire_sql hires 
              inner join dfBio_sql bio on bio.id = hires.id
              Where hires.hired = 1
              group by bio.service_branch__c
              order by hired asc
              """).show()

#Not hired by Service Branch
spark.sql(""" Select count(hires.id) as Not_hired,bio.service_branch__c 
              From dfHire_sql hires 
              inner join dfBio_sql bio on bio.id = hires.id
              Where hires.hired = 0
              group by bio.service_branch__c
              order by Not_hired asc
              """).show()

#Hired by rank
spark.sql(""" Select count(hires.id) as hired,bio.service_rank__c
              From dfHire_sql hires 
              inner join dfBio_sql bio on bio.id = hires.id
              Where hires.hired = 1
              group by bio.service_rank__c
              order by hired desc
              """).show()

#Prefered method of contact for the hired
spark.sql(""" select count(com.id) as count, com.preferred_method_of_contact__c 
              from dfCom_sql com 
              inner join dfHire_sql hires on com.id = hires.id
              where hires.hired = 1
              group by com.preferred_method_of_contact__c 
              order by count
          """).show()


#Prefered method of contact for the Unhired
spark.sql(""" select count(com.id) as count, com.preferred_method_of_contact__c 
              from dfCom_sql com 
              inner join dfHire_sql hires on com.id = hires.id
              where hires.hired = 0
              group by com.preferred_method_of_contact__c 
              order by count
          """).show()

#Select variables that determine whether one is going to be employed or not, store those varibles in a new dataframe
dfspark= spark.sql(""" select fact.yearsinservice as S_years , fact.reg_afterservice_years as R_years,
               act.resume_tailoring_tips__c as Resume_Tips,act.finalized_hhusa_revised_resume_on_file__c as Resume_OnFile, bio.client__c as Client, bio.client_type__c as Client_Type, bio.service_rank__c as Service_R
              ,bio.service_branch__c as Service_B, hire.hire_heroes_usa_confirmed_hire__c as HHUSA_hire, com.preferred_method_of_contact__c as Com_Method, com.responsive__c
              ,com.active__c, act.created_linkedin_account__c as Created_Linkedin, bio.highest_level_of_education_completed__c as Educ,  hire.hired
              ,bio.primary_military_occupational_specialty__c as Occupation
               
              from dfFact_sql fact
              
              inner join dfAct_sql act on act.id = fact.id
              inner join dfBio_sql bio on bio.id = fact.id
              inner join dfHire_sql hire on hire.id = fact.id
              inner join dfCom_sql com on com.id = fact.id
              
              where bio.client_type__c != '' and bio.service_branch__c !='' and bio.service_rank__c !='' 
              and com.preferred_method_of_contact__c !='' and fact.yearsinservice < 53 and fact.yearsinservice > 0 
              and fact.reg_afterservice_years >- 32 and fact.reg_afterservice_years < 53 and com.active__c !=''
              and bio.highest_level_of_education_completed__c !='' and hire.hired  !='' and hire.hired !='No' and hire.hired != 3

          """)

dfsparkcopy = dfspark

# Get the number of clients in the description above
dfspark.count()



# Change the data type of the the hired column from string to double
dfspark = dfspark.withColumn("hired", dfspark["hired"].cast(DoubleType()))

# Confirm the labels in the hired column 0 represents the client was not hired 1 represents they were hired. 
dfspark.select('hired').distinct().show()


# The data types of the data frame 
dfspark.dtypes

#Hash the features through a hash function to convert the features into vectors for the dfspark dataframe,
# Convert strings to numerical data, the output column is features. The target is not included in this conversion.
funchash = FeatureHasher(inputCols= ['S_years','R_years','Resume_Tips','Resume_OnFile','Client','Client_Type','Service_R',\
 'Service_B','HHUSA_hire','Com_Method','responsive__c','active__c','Created_Linkedin','Educ','Occupation'],
                       outputCol="features")

#Add the new added features to the dataframe
dffeatures = funchash.transform(dfspark)


#Confirm that the dataframe is active, We are going to use this data to train our machine learning algorithms.
dffeatures.select('features','hired').show()

#Split the data into training and testing data for the training of the algorithms and testing of the algorithms 
train_data,test_data=dffeatures.randomSplit([0.75,0.25])


# Train the decision tree classifier, to predict whether one is hired or not, using the hired column and features. 
def treedec(label, features):
    decision_tree = DecisionTreeClassifier(labelCol=label, featuresCol=features)
    return decision_tree


def traindec_tree(train_data, algo): 
    # The model 
    model1 = algo.fit(train_data)
    return model1

# Make predictions using the test set of the dataset ... Store the predictions in a new dataframe prediction 
predictions = traindec_tree(train_data, treedec('hired','features')).transform(test_data)

# Check out the new dataframe predictions to view the prediction, hired and features columns 
predictions.select("prediction", "hired", "features").show()

# Now lets evaluate the algorithm performance
def evaluate_algo(df,label,Algorithm_name  ):
    classification_evaluator = MulticlassClassificationEvaluator(
    labelCol=label, predictionCol="prediction")
    accuracy = classification_evaluator.evaluate(df)
    print(Algorithm_name )
    print("Test Error  = %g " % (1.0 - accuracy))
    print("Accuracy   = %g " % accuracy)

print(" ")
evaluate_algo(predictions,'hired','Decision Tree')


#Random forest
def randomforest(label, features, numTrees):
    """ takes in labels, features and number of trees"""
    random_forest = RandomForestClassifier(labelCol=label, featuresCol=features, numTrees=numTrees)
    
    return random_forest

#Random forest fit function 
def random_forestfit(randf,train_data):
    """ Takes in a dataframe, and a random forest model """
    model= randf.fit(train_data)
    return model
 

# Make predictions.
predictions2 = random_forestfit(randomforest('hired', 'features', 17),train_data).transform(test_data)

# Select example rows to display.
predictions2.select('prediction','hired','features').show()

print(" ")
evaluate_algo(predictions2,'hired','Random Forest')

#Clustering
#Hash the features through a hash function to convert the features into vectors for the dfspark dataframe,
# Convert strings to numerical data, the output column is features. The target is not included in this conversion.
funchash = FeatureHasher(inputCols= ['S_years','R_years','Resume_Tips','Resume_OnFile','Client','Client_Type','Service_R',\
 'Service_B','HHUSA_hire','Com_Method','responsive__c','active__c','Created_Linkedin','Educ','Occupation'],
                       outputCol="features")

dffeatures = funchash.transform(dfspark)
dffeatures.persist()
dffeatures.show(1)
def cluster(k,df,features):
    """ Takes in k as an int, the dataframe, and features """
    kmeans = KMeans(k=k, seed=1)  # 2 clusters here
    model = kmeans.fit(df.select(features))
    return model

transformeddf = cluster(4,dffeatures,'features').transform(dffeatures)

#Number of clusters
print(" ")
print ("Number of clusters")
transformeddf.select('prediction').distinct().show()

#Service branch with Service Rank, Showing whether hired or not. 
print(" ")
print(" Service branches with Service Ranks and cluster as prediction")
transformeddf.select('Service_B','Service_R','hired','prediction').distinct().show()


#Information about individual clusters in term of years service men stay in the army and when they register for services
print(" ")
print ("Statistical Information about years in service and Registration for services with HHUSA")
transformeddf.select('S_years','R_years').describe().show()

#Get cluster information Cluster 0 in terms of years service men spend on service.
print(" ")
print ("Years clustered with cluster 0 ")
cluster0 = transformeddf.filter(transformeddf.prediction==0)
cluster0.select('S_years','R_years').describe().show()

#Get cluster information Cluster 1
print(" ")
print ("Years clustered with cluster 1 ")
cluster1 = transformeddf.filter(transformeddf.prediction==1)
cluster1.select('S_years','R_years').describe().show()

#Get cluster information Cluster 2
print(" ")
print ("Years clustered with cluster 2")
cluster2 = transformeddf.filter(transformeddf.prediction==2)
cluster2.select('S_years','R_years').describe().show()


#Get cluster information Cluster 3
print(" ")
print ("Years clustered with cluster 3 ")
cluster3 = transformeddf.filter(transformeddf.prediction==3)
cluster3.select('S_years','R_years').describe().show()


end = datetime.now()

print('Duration of script: {}'.format(end - start))

Overwriting largescale.py


## Text mining

In [3]:
from pyspark.ml.feature import Tokenizer, RegexTokenizer

In [4]:
textdf = spark.read.json("/home/mose/Downloads/Train_onetag_small.json")

# The persist method saves resources in terms of memory to be used for the dataframe 
textdf.persist()

DataFrame[Body: string, Id: bigint, Tags: string, Title: string, oneTag: string]

In [5]:
textdf.columns

['Body', 'Id', 'Tags', 'Title', 'oneTag']

In [6]:
textdf.head(1)

[Row(Body="<p>I'd like to check if an uploaded file is an image file (e.g png, jpg, jpeg, gif, bmp) or another file. The problem is that I'm using Uploadify to upload the files, which changes the mime type and gives a 'text/octal' or something as the mime type, no matter which file type you upload.</p>\n\n<p>Is there a way to check if the uploaded file is an image apart from checking the file extension using PHP?</p>\n", Id=1, Tags='php image-processing file-upload upload mime-types', Title='How to check if an uploaded file is an image without mime type?', oneTag='php')]

In [7]:
#Coverts sentences into words takes into context regular expression patters 
RegTokenizer = RegexTokenizer(inputCol="Body", outputCol="Words", pattern="\\W")
textdf = RegTokenizer.transform(textdf)

In [8]:
textdf.head()

Row(Body="<p>I'd like to check if an uploaded file is an image file (e.g png, jpg, jpeg, gif, bmp) or another file. The problem is that I'm using Uploadify to upload the files, which changes the mime type and gives a 'text/octal' or something as the mime type, no matter which file type you upload.</p>\n\n<p>Is there a way to check if the uploaded file is an image apart from checking the file extension using PHP?</p>\n", Id=1, Tags='php image-processing file-upload upload mime-types', Title='How to check if an uploaded file is an image without mime type?', oneTag='php', Words=['p', 'i', 'd', 'like', 'to', 'check', 'if', 'an', 'uploaded', 'file', 'is', 'an', 'image', 'file', 'e', 'g', 'png', 'jpg', 'jpeg', 'gif', 'bmp', 'or', 'another', 'file', 'the', 'problem', 'is', 'that', 'i', 'm', 'using', 'uploadify', 'to', 'upload', 'the', 'files', 'which', 'changes', 'the', 'mime', 'type', 'and', 'gives', 'a', 'text', 'octal', 'or', 'something', 'as', 'the', 'mime', 'type', 'no', 'matter', 'which

In [9]:
#Get the length of the body or the body 
lenbody = udf(lambda x:len(x), IntegerType())
textdf = textdf.withColumn("LenBody",lenbody(textdf.Words))

In [10]:
textdf.head(2)

[Row(Body="<p>I'd like to check if an uploaded file is an image file (e.g png, jpg, jpeg, gif, bmp) or another file. The problem is that I'm using Uploadify to upload the files, which changes the mime type and gives a 'text/octal' or something as the mime type, no matter which file type you upload.</p>\n\n<p>Is there a way to check if the uploaded file is an image apart from checking the file extension using PHP?</p>\n", Id=1, Tags='php image-processing file-upload upload mime-types', Title='How to check if an uploaded file is an image without mime type?', oneTag='php', Words=['p', 'i', 'd', 'like', 'to', 'check', 'if', 'an', 'uploaded', 'file', 'is', 'an', 'image', 'file', 'e', 'g', 'png', 'jpg', 'jpeg', 'gif', 'bmp', 'or', 'another', 'file', 'the', 'problem', 'is', 'that', 'i', 'm', 'using', 'uploadify', 'to', 'upload', 'the', 'files', 'which', 'changes', 'the', 'mime', 'type', 'and', 'gives', 'a', 'text', 'octal', 'or', 'something', 'as', 'the', 'mime', 'type', 'no', 'matter', 'whic

In [11]:
# The re python library finds all regular expresions that are indicated
numparagraphs = udf(lambda x: len(re.findall("</p>",x)),IntegerType())
numlinks = udf(lambda x: len(re.findall("</a>",x)), IntegerType())

In [12]:
textdf = textdf.withColumn("numParagraphs",numparagraphs(textdf.Body))

In [13]:
textdf = textdf.withColumn("Linksnum", numlinks(textdf.Body))

In [14]:
textdf.select("numParagraphs","Linksnum").show()

+-------------+--------+
|numParagraphs|Linksnum|
+-------------+--------+
|            2|       0|
|            2|       0|
|            4|       0|
|            7|       1|
|            2|       0|
|            1|       0|
|            9|       0|
|            4|       0|
|            4|       0|
|            3|       0|
|            4|       1|
|            4|       0|
|            1|       0|
|            2|       0|
|            1|       0|
|           12|       0|
|            4|       0|
|            3|       0|
|            4|       1|
|            8|       0|
+-------------+--------+
only showing top 20 rows



In [15]:
textdf.columns

['Body',
 'Id',
 'Tags',
 'Title',
 'oneTag',
 'Words',
 'LenBody',
 'numParagraphs',
 'Linksnum']

In [16]:
# We can now put our newly created features into a vector using the vectorizer function
assembler = VectorAssembler(inputCols=["LenBody","numParagraphs","Linksnum"],outputCol="features")
textdf = assembler.transform(textdf)

In [17]:
# Scale the data uniformly
scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures")

In [18]:
scalerModel = scaler.fit(textdf)
textdf = scalerModel.transform(textdf)

In [19]:
textdf.select('scaledFeatures').show(1, False)

+--------------------------------------------+
|scaledFeatures                              |
+--------------------------------------------+
|[0.43245858153237177,0.7036973726344203,0.0]|
+--------------------------------------------+
only showing top 1 row



In [20]:
textdf.head(1)

[Row(Body="<p>I'd like to check if an uploaded file is an image file (e.g png, jpg, jpeg, gif, bmp) or another file. The problem is that I'm using Uploadify to upload the files, which changes the mime type and gives a 'text/octal' or something as the mime type, no matter which file type you upload.</p>\n\n<p>Is there a way to check if the uploaded file is an image apart from checking the file extension using PHP?</p>\n", Id=1, Tags='php image-processing file-upload upload mime-types', Title='How to check if an uploaded file is an image without mime type?', oneTag='php', Words=['p', 'i', 'd', 'like', 'to', 'check', 'if', 'an', 'uploaded', 'file', 'is', 'an', 'image', 'file', 'e', 'g', 'png', 'jpg', 'jpeg', 'gif', 'bmp', 'or', 'another', 'file', 'the', 'problem', 'is', 'that', 'i', 'm', 'using', 'uploadify', 'to', 'upload', 'the', 'files', 'which', 'changes', 'the', 'mime', 'type', 'and', 'gives', 'a', 'text', 'octal', 'or', 'something', 'as', 'the', 'mime', 'type', 'no', 'matter', 'whic

In [21]:
# Setting the word count and the number of words we want to keep. In this case there are 1500
cv = CountVectorizer(inputCol="Words", outputCol="WordCount2", vocabSize=1500)
model = cv.fit(textdf)
textdf = model.transform(textdf)

In [22]:
textdf.columns

['Body',
 'Id',
 'Tags',
 'Title',
 'oneTag',
 'Words',
 'LenBody',
 'numParagraphs',
 'Linksnum',
 'features',
 'scaledFeatures',
 'WordCount2']

In [23]:
textdf.select('WordCount2').show(1,False)

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|WordCount2                                                                                                                                                                                                                                                                                                                           |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|(1500,[0,1,2,3,

In [24]:
textdf.count()

100000

In [25]:
# The most common words in the document
model.vocabulary

['p',
 'the',
 'i',
 'to',
 'code',
 'a',
 'gt',
 'lt',
 'is',
 'and',
 'pre',
 'in',
 'this',
 'of',
 'it',
 'that',
 'for',
 '0',
 '1',
 'have',
 'my',
 'if',
 'on',
 'but',
 'with',
 'can',
 'not',
 'be',
 'as',
 't',
 'li',
 'from',
 '2',
 's',
 'http',
 'an',
 'm',
 'strong',
 'new',
 'how',
 'do',
 'com',
 'so',
 'or',
 'at',
 'using',
 'when',
 'am',
 'like',
 'class',
 'id',
 'there',
 'get',
 'are',
 'name',
 'what',
 'any',
 'file',
 'string',
 'data',
 'all',
 'which',
 'want',
 'would',
 'amp',
 'use',
 'java',
 'function',
 'public',
 'some',
 '3',
 'text',
 'error',
 'android',
 'value',
 'c',
 'x',
 'href',
 'you',
 'one',
 'by',
 'user',
 'me',
 'server',
 'type',
 'here',
 'way',
 'return',
 'int',
 'will',
 'div',
 'need',
 'then',
 'set',
 'e',
 'system',
 'has',
 'problem',
 'out',
 'php',
 'no',
 'just',
 '4',
 'org',
 'know',
 'html',
 'only',
 'where',
 'page',
 'application',
 '5',
 'thanks',
 'var',
 'br',
 'we',
 'd',
 'should',
 'does',
 'add',
 'n',
 'true',

In [26]:
#Least common words 
model.vocabulary[-5:]

['provided', 'against', 'expect', 'focus', 'strange']

In [27]:
# Tidf is a numerical statistic that is intended to reflect how important a word is to a document in a collection or corpus.[1]
# Word frequency in comparison to the whole document. 
idf = IDF(inputCol="WordCount2", outputCol="tfidf")
idfModel = idf.fit(textdf)
textdf = idfModel.transform(textdf)

In [28]:
textdf.head()

Row(Body="<p>I'd like to check if an uploaded file is an image file (e.g png, jpg, jpeg, gif, bmp) or another file. The problem is that I'm using Uploadify to upload the files, which changes the mime type and gives a 'text/octal' or something as the mime type, no matter which file type you upload.</p>\n\n<p>Is there a way to check if the uploaded file is an image apart from checking the file extension using PHP?</p>\n", Id=1, Tags='php image-processing file-upload upload mime-types', Title='How to check if an uploaded file is an image without mime type?', oneTag='php', Words=['p', 'i', 'd', 'like', 'to', 'check', 'if', 'an', 'uploaded', 'file', 'is', 'an', 'image', 'file', 'e', 'g', 'png', 'jpg', 'jpeg', 'gif', 'bmp', 'or', 'another', 'file', 'the', 'problem', 'is', 'that', 'i', 'm', 'using', 'uploadify', 'to', 'upload', 'the', 'files', 'which', 'changes', 'the', 'mime', 'type', 'and', 'gives', 'a', 'text', 'octal', 'or', 'something', 'as', 'the', 'mime', 'type', 'no', 'matter', 'which

In [29]:
textdf.columns

['Body',
 'Id',
 'Tags',
 'Title',
 'oneTag',
 'Words',
 'LenBody',
 'numParagraphs',
 'Linksnum',
 'features',
 'scaledFeatures',
 'WordCount2',
 'tfidf']

In [30]:
#The target, we need to convert it to numerical 
textdf.select('oneTag').show()

+----------------+
|          oneTag|
+----------------+
|             php|
|         firefox|
|               r|
|              c#|
|             php|
|active-directory|
|           other|
|              c#|
|      javascript|
|             sql|
|            .net|
|       algorithm|
|           other|
|   documentation|
|       windows-7|
|             php|
|               r|
|             wpf|
|      javascript|
|             php|
+----------------+
only showing top 20 rows



In [31]:
# Convert the oneTag to index
indexer = StringIndexer(inputCol='oneTag', outputCol="oneTagIndex")
textdf = indexer.fit(textdf).transform(textdf)

In [32]:
textdf.select('OneTagIndex').show(5,False)

+-----------+
|OneTagIndex|
+-----------+
|3.0        |
|91.0       |
|29.0       |
|1.0        |
|3.0        |
+-----------+
only showing top 5 rows



In [33]:
textdf.filter(textdf.Id==115).select('LenBody').show()

+-------+
|LenBody|
+-------+
|     12|
+-------+



In [34]:
textdf.columns

['Body',
 'Id',
 'Tags',
 'Title',
 'oneTag',
 'Words',
 'LenBody',
 'numParagraphs',
 'Linksnum',
 'features',
 'scaledFeatures',
 'WordCount2',
 'tfidf',
 'oneTagIndex']

In [35]:
#Get the most important features of the TFIDF 
#pca = PCA(k=2, inputCol="features", outputCol="pca_features")
#model = pca.fit(df)
#model.explainedVariance
pca = PCA(k=120, inputCol="tfidf", outputCol="pcaFeatures")
modelpca = pca.fit(textdf)

In [36]:
textdf = modelpca.transform(textdf)

In [37]:
textdf.head()

Row(Body="<p>I'd like to check if an uploaded file is an image file (e.g png, jpg, jpeg, gif, bmp) or another file. The problem is that I'm using Uploadify to upload the files, which changes the mime type and gives a 'text/octal' or something as the mime type, no matter which file type you upload.</p>\n\n<p>Is there a way to check if the uploaded file is an image apart from checking the file extension using PHP?</p>\n", Id=1, Tags='php image-processing file-upload upload mime-types', Title='How to check if an uploaded file is an image without mime type?', oneTag='php', Words=['p', 'i', 'd', 'like', 'to', 'check', 'if', 'an', 'uploaded', 'file', 'is', 'an', 'image', 'file', 'e', 'g', 'png', 'jpg', 'jpeg', 'gif', 'bmp', 'or', 'another', 'file', 'the', 'problem', 'is', 'that', 'i', 'm', 'using', 'uploadify', 'to', 'upload', 'the', 'files', 'which', 'changes', 'the', 'mime', 'type', 'and', 'gives', 'a', 'text', 'octal', 'or', 'something', 'as', 'the', 'mime', 'type', 'no', 'matter', 'which

In [29]:
from pyspark.ml.feature import OneHotEncoder

df = spark.createDataFrame([
    (0.0, 1.0),
    (1.0, 0.0),
    (2.0, 1.0),
    (0.0, 2.0),
    (0.0, 1.0),
    (2.0, 0.0)
], ["categoryIndex1", "categoryIndex2"])

encoder = OneHotEncoder(inputCols=["categoryIndex1", "categoryIndex2"],
                        outputCols=["categoryVec1", "categoryVec2"])
model = encoder.fit(df)
encoded = model.transform(df)
encoded.show()

+--------------+--------------+-------------+-------------+
|categoryIndex1|categoryIndex2| categoryVec1| categoryVec2|
+--------------+--------------+-------------+-------------+
|           0.0|           1.0|(2,[0],[1.0])|(2,[1],[1.0])|
|           1.0|           0.0|(2,[1],[1.0])|(2,[0],[1.0])|
|           2.0|           1.0|    (2,[],[])|(2,[1],[1.0])|
|           0.0|           2.0|(2,[0],[1.0])|    (2,[],[])|
|           0.0|           1.0|(2,[0],[1.0])|(2,[1],[1.0])|
|           2.0|           0.0|    (2,[],[])|(2,[0],[1.0])|
+--------------+--------------+-------------+-------------+



In [30]:
dfspark

DataFrame[S_years: double, R_years: double, Resume_Tips: int, Resume_OnFile: int, Client: int, Client_Type: string, Service_R: string, Service_B: string, HHUSA_hire: int, Com_Method: string, responsive__c: int, active__c: string, Created_Linkedin: int, Educ: string, hired: double, Occupation: string]

In [32]:
from pyspark.ml.feature import StringIndexer


indexer = StringIndexer(inputCol="Client_Type", outputCol="Client_TypeIndex")
dfspark = indexer.fit(dfspark).transform(dfspark)
dfspark.

In [33]:
encoder = OneHotEncoder(inputCols=["Client_TypeIndex"],
                        outputCols=["Client_TypeIndexVec"])
model = encoder.fit(dfspark)
dfspark = model.transform(dfspark)

In [34]:
dfspark

DataFrame[S_years: double, R_years: double, Resume_Tips: int, Resume_OnFile: int, Client: int, Client_Type: string, Service_R: string, Service_B: string, HHUSA_hire: int, Com_Method: string, responsive__c: int, active__c: string, Created_Linkedin: int, Educ: string, hired: double, Occupation: string, Client_TypeIndex: double, Client_TypeIndexVec: vector]

In [39]:
dfspark.select('Client_Type','Client_TypeIndexVec').head()

Row(Client_Type='Online Registrant', Client_TypeIndexVec=SparseVector(4, {0: 1.0}))

In [56]:
dfspark.dtypes

[('S_years', 'double'),
 ('R_years', 'double'),
 ('Resume_Tips', 'int'),
 ('Resume_OnFile', 'int'),
 ('Client', 'int'),
 ('Client_Type', 'string'),
 ('Service_R', 'string'),
 ('Service_B', 'string'),
 ('HHUSA_hire', 'int'),
 ('Com_Method', 'string'),
 ('responsive__c', 'int'),
 ('active__c', 'string'),
 ('Created_Linkedin', 'int'),
 ('Educ', 'string'),
 ('hired', 'double'),
 ('Occupation', 'string'),
 ('Client_TypeIndex', 'double'),
 ('Client_TypeIndexVec', 'vector')]

In [57]:
dfspark = dfspark.drop('Client_TypeIndex')
dfspark = dfspark.drop('Client_TypeIndexVec')

In [58]:
dfspark.columns

['S_years',
 'R_years',
 'Resume_Tips',
 'Resume_OnFile',
 'Client',
 'Client_Type',
 'Service_R',
 'Service_B',
 'HHUSA_hire',
 'Com_Method',
 'responsive__c',
 'active__c',
 'Created_Linkedin',
 'Educ',
 'hired',
 'Occupation']

In [20]:
from pyspark.ml import Pipeline

indexers = [StringIndexer(inputCol=i, outputCol=i+"Index").fit(dfspark) \
            for i in list(set(dfspark.columns)- set(['S_years','R_years','Resume_Tips','Resume_OnFile','Client','HHUSA_hire','responsive__c','Created_Linkedin','Occupation','Educ'])) ]
pipeline = Pipeline(stages=indexers)
dfspark = pipeline.fit(dfspark).transform(dfspark)

In [21]:
dfspark

DataFrame[S_years: double, R_years: double, Resume_Tips: int, Resume_OnFile: int, Client: int, Client_Type: string, Service_R: string, Service_B: string, HHUSA_hire: int, Com_Method: string, responsive__c: int, active__c: string, Created_Linkedin: int, Educ: string, hired: double, Occupation: string, Com_MethodIndex: double, hiredIndex: double, active__cIndex: double, Client_TypeIndex: double, Service_BIndex: double, Service_RIndex: double]

In [61]:
dfspark.dtypes

[('S_years', 'double'),
 ('R_years', 'double'),
 ('Resume_Tips', 'int'),
 ('Resume_OnFile', 'int'),
 ('Client', 'int'),
 ('Client_Type', 'string'),
 ('Service_R', 'string'),
 ('Service_B', 'string'),
 ('HHUSA_hire', 'int'),
 ('Com_Method', 'string'),
 ('responsive__c', 'int'),
 ('active__c', 'string'),
 ('Created_Linkedin', 'int'),
 ('Educ', 'string'),
 ('hired', 'double'),
 ('Occupation', 'string'),
 ('Service_BIndex', 'double'),
 ('Com_MethodIndex', 'double'),
 ('active__cIndex', 'double'),
 ('Service_RIndex', 'double'),
 ('Client_TypeIndex', 'double'),
 ('hiredIndex', 'double')]

In [22]:
from pyspark.ml.feature import OneHotEncoder
encoder = OneHotEncoder(inputCols=["Service_BIndex", "Com_MethodIndex","active__cIndex", "Service_RIndex", "Client_TypeIndex"],
                       outputCols=["Service_BIndexVec", "Com_MethodIndexVec","active__cIndexVec", "Service_RIndexVec", "Client_TypeIndexVec"])
modelOne = encoder.fit(dfspark)
dfspark = modelOne.transform(dfspark)

In [23]:
# Learn a mapping from words to Vectors.
from pyspark.ml.feature import Word2Vec
from pyspark.sql.functions import split, regexp_replace


dfspark = dfspark.withColumn(
    "Educ",
    split(regexp_replace("Educ", r"(^\[\[\[)|(\]\]\]$)", ""), ", ")
)

#word2Vec = Word2Vec(minCount=0, inputCol="Educ", outputCol="EducVec")
#modelw = word2Vec.fit(dfspark)

#dfspark = modelw.transform(dfspark)


In [24]:
dfspark.head()

Row(S_years=8.11, R_years=-0.37, Resume_Tips=1, Resume_OnFile=1, Client=1, Client_Type='Online Registrant', Service_R='E-5', Service_B='Army', HHUSA_hire=0, Com_Method='Telephone', responsive__c=1, active__c='Yes', Created_Linkedin=1, Educ=['High School/GED'], hired=0.0, Occupation='91F', Com_MethodIndex=0.0, hiredIndex=0.0, active__cIndex=0.0, Client_TypeIndex=0.0, Service_BIndex=0.0, Service_RIndex=1.0, active__cIndexVec=SparseVector(1, {0: 1.0}), Service_RIndexVec=SparseVector(25, {1: 1.0}), Client_TypeIndexVec=SparseVector(4, {0: 1.0}), Com_MethodIndexVec=SparseVector(4, {0: 1.0}), Service_BIndexVec=SparseVector(4, {0: 1.0}))

In [25]:
dfspark = dfspark.withColumn(
    "Occupation",
    split(regexp_replace("Occupation", r"(^\[\[\[)|(\]\]\]$)", ""), ", ")
)

In [25]:
word2Vec = Word2Vec(minCount=0, inputCol="Educ", outputCol="EducVec")
modelw = word2Vec.fit(dfspark)
dfspark = modelw.transform(dfspark)

In [86]:
dfspark.dtypes

[('S_years', 'double'),
 ('R_years', 'double'),
 ('Resume_Tips', 'int'),
 ('Resume_OnFile', 'int'),
 ('Client', 'int'),
 ('Client_Type', 'string'),
 ('Service_R', 'string'),
 ('Service_B', 'string'),
 ('HHUSA_hire', 'int'),
 ('Com_Method', 'string'),
 ('responsive__c', 'int'),
 ('active__c', 'string'),
 ('Created_Linkedin', 'int'),
 ('Educ', 'array<string>'),
 ('hired', 'double'),
 ('Occupation', 'array<string>'),
 ('Service_BIndex', 'double'),
 ('Com_MethodIndex', 'double'),
 ('active__cIndex', 'double'),
 ('Service_RIndex', 'double'),
 ('Client_TypeIndex', 'double'),
 ('hiredIndex', 'double'),
 ('active__cIndexVec', 'vector'),
 ('Service_RIndexVec', 'vector'),
 ('Client_TypeIndexVec', 'vector'),
 ('Com_MethodIndexVec', 'vector'),
 ('Service_BIndexVec', 'vector'),
 ('EducVec', 'vector')]

In [26]:
# We can now put our newly created features into a vector using the vectorizer function
assembler2 = VectorAssembler(inputCols=['Resume_Tips','Resume_OnFile','Client','HHUSA_hire','responsive__c','Created_Linkedin','active__cIndexVec','Service_RIndexVec','Client_TypeIndexVec','Com_MethodIndexVec','Service_BIndexVec','EducVec'],outputCol="featurevect")
dfspark = assembler2.transform(dfspark)

In [29]:
dfspark.head()

Row(S_years=8.11, R_years=-0.37, Resume_Tips=1, Resume_OnFile=1, Client=1, Client_Type='Online Registrant', Service_R='E-5', Service_B='Army', HHUSA_hire=0, Com_Method='Telephone', responsive__c=1, active__c='Yes', Created_Linkedin=1, Educ=['High School/GED'], hired='0', Occupation=['91F'], Service_BIndex=0.0, Client_TypeIndex=0.0, Service_RIndex=1.0, active__cIndex=0.0, hiredIndex=0.0, Com_MethodIndex=0.0, active__cIndexVec=SparseVector(1, {0: 1.0}), Service_RIndexVec=SparseVector(25, {1: 1.0}), Client_TypeIndexVec=SparseVector(4, {0: 1.0}), Com_MethodIndexVec=SparseVector(4, {0: 1.0}), Service_BIndexVec=SparseVector(4, {0: 1.0}), EducVec=DenseVector([0.0006, -0.0002, 0.0045, -0.0022, 0.004, 0.0033, 0.0028, 0.003, 0.0011, 0.0044, -0.0049, -0.0005, 0.0039, 0.0, -0.0021, 0.0007, -0.0027, 0.0036, 0.0039, -0.0022, -0.0043, 0.0024, 0.0023, 0.0044, 0.0043, 0.0042, -0.0044, 0.0008, -0.001, 0.0032, 0.0028, -0.0008, -0.0008, 0.0034, 0.0012, 0.0013, 0.0039, -0.0032, 0.0011, 0.0027, 0.001, 0.001

In [27]:
dfspark.select('featurevect','hired').show()

+--------------------+-----+
|         featurevect|hired|
+--------------------+-----+
|[1.0,1.0,1.0,0.0,...|  0.0|
|[1.0,1.0,1.0,1.0,...|  0.0|
|[1.0,1.0,1.0,0.0,...|  0.0|
|[1.0,1.0,1.0,0.0,...|  0.0|
|[0.0,1.0,1.0,0.0,...|  0.0|
|[1.0,1.0,1.0,0.0,...|  0.0|
|[1.0,1.0,1.0,0.0,...|  0.0|
|[1.0,1.0,1.0,0.0,...|  0.0|
|[1.0,1.0,1.0,0.0,...|  0.0|
|[0.0,0.0,1.0,0.0,...|  0.0|
|[1.0,1.0,1.0,0.0,...|  0.0|
|[1.0,1.0,1.0,0.0,...|  0.0|
|[1.0,1.0,1.0,0.0,...|  0.0|
|[1.0,1.0,1.0,1.0,...|  0.0|
|[1.0,1.0,1.0,0.0,...|  0.0|
|[1.0,1.0,1.0,1.0,...|  0.0|
|[0.0,1.0,1.0,1.0,...|  0.0|
|[1.0,1.0,1.0,0.0,...|  0.0|
|[0.0,0.0,1.0,0.0,...|  0.0|
|[1.0,1.0,1.0,1.0,...|  0.0|
+--------------------+-----+
only showing top 20 rows



In [28]:
# Scale the data uniformly
scaler2 = StandardScaler(inputCol="featurevect", outputCol="features")
scalerModel = scaler2.fit(dfspark)
dfspark = scalerModel.transform(dfspark)

In [29]:
dfspark.select('hired','features').head()

Row(hired=0.0, features=DenseVector([2.1702, 2.5864, 0.0, 0.0, 2.8238, 2.0136, 9.2445, 0.0, 2.4497, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.8913, 0.0, 0.0, 0.0, 2.5965, 0.0, 0.0, 0.0, 2.009, 0.0, 0.0, 0.0, 0.0635, -0.0142, 0.0689, 0.0125, 0.0133, -0.0059, -0.0231, -0.0398, -0.0155, 0.0026, -0.0724, 0.0363, 0.0434, 0.0286, -0.0137, 0.0389, 0.0176, 0.0234, -0.0579, 0.0149, 0.0188, -0.0503, -0.0475, -0.0449, -0.0004, 0.0037, -0.0566, 0.0056, 0.011, 0.0443, 0.0202, -0.0016, -0.0067, 0.0171, -0.0024, -0.0271, -0.052, -0.0147, 0.0333, 0.021, -0.0429, -0.0355, 0.0139, -0.0197, 0.0208, -0.0595, -0.0119, 0.0048, -0.0022, 0.0154, -0.0549, 0.001, 0.0804, -0.0033, 0.0231, -0.0235, 0.0158, 0.0047, 0.0052, -0.0126, -0.0358, -0.0079, -0.0491, -0.0562, -0.018, 0.0331, 0.04, 0.0533, -0.0221, 0.0346, -0.0399, 0.0022, -0.0334, 0.0062, -0.0086, 0.0203, -0.0935, -0.0296, 0.0044, 0.17, 0.0133, 0.0212, -0.0142, -0.027, -0.0061, 0.11

In [40]:
# Change the data type of the the hired column from string to double
dfspark = dfspark.withColumn("hired", dfspark["hired"].cast(DoubleType()))

In [30]:
train_data,test_data=dfspark.select('features','hired').randomSplit([0.7,0.3])

In [31]:
decision_tree = DecisionTreeClassifier(labelCol='hired', featuresCol='features')

In [32]:
model1 = decision_tree.fit(train_data)

In [33]:
pred = model1.transform(test_data)

In [34]:
pred.head()

Row(features=DenseVector([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.2445, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0881, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.8913, 0.0, 0.0, 0.0, 2.5965, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.9716, -0.2374, -1.8123, 1.1851, 1.0588, 1.6715, 0.8131, -1.0705, -0.473, -0.7603, 0.6095, -1.175, 1.3379, 1.064, 1.102, 0.2272, 1.7169, -1.4341, -1.0796, 0.0412, 1.4236, 1.3448, 0.9022, 0.0683, -1.1246, 1.7862, -1.6821, -2.1687, -1.8141, 1.4642, -2.0638, 0.9235, -0.6705, 0.3531, -0.5954, -0.2825, 1.3547, -0.2076, -0.6091, -0.5387, -1.433, 1.6622, -1.6802, -0.6257, -1.5959, -0.5235, -0.007, -1.0847, 2.0039, 1.5939, 1.364, 1.217, -0.8326, -0.2589, -0.6978, -0.6373, -1.1174, -1.7657, 0.9841, 1.4137, -1.1052, 1.1403, -1.4928, 0.4257, -0.6137, 1.5075, 1.8168, 0.0113, 1.7317, -1.907, 0.5285, 0.8976, -1.5635, 0.843, -0.4441, -0.9456, 0.5735, -1.8887, -1.0578, -0.1059, -2.1784, 1.1683, 0.4057, -0.0122, 1.031, 1.1531, 1.877, -0.0493, -1.2581, 

In [35]:
pred.columns

['features', 'hired', 'rawPrediction', 'probability', 'prediction']

In [36]:
evaluate_algo(pred,'hired','Decision Tree')

NameError: name 'evaluate_algo' is not defined

In [37]:
random_forest = RandomForestClassifier(labelCol='hired', featuresCol='features', numTrees=28)
model2= random_forest.fit(train_data)
predrand = model2.transform(test_data)

In [117]:
predrand.columns

['scaledFeatures', 'hired', 'rawPrediction', 'probability', 'prediction']

In [118]:
evaluate_algo(predrand,'hired','Random Tree')

Random Tree
Test Error  = 0.274526 
Accuracy   = 0.725474 


In [38]:
dfspark.select('features').show()

+--------------------+
|            features|
+--------------------+
|[2.17015087637283...|
|[2.17015087637283...|
|[2.17015087637283...|
|[2.17015087637283...|
|[0.0,2.5863992700...|
|[2.17015087637283...|
|[2.17015087637283...|
|[2.17015087637283...|
|[2.17015087637283...|
|[0.0,0.0,0.0,0.0,...|
|[2.17015087637283...|
|[2.17015087637283...|
|[2.17015087637283...|
|[2.17015087637283...|
|[2.17015087637283...|
|[2.17015087637283...|
|[0.0,2.5863992700...|
|[2.17015087637283...|
|[0.0,0.0,0.0,0.0,...|
|[2.17015087637283...|
+--------------------+
only showing top 20 rows



In [38]:
# Change the data type of the the hired column from string to double
dfspark = dfspark.withColumn("hired", dfspark["hired"].cast(DoubleType()))

In [49]:
b = dfspark.select('scaledFeatures')
kmeans2 = KMeans(k=4, seed=1)  # 2 clusters here
modelk2 = kmeans2.fit(b)
#dfspark = modelk.transform(dfspark)

IllegalArgumentException: features does not exist. Available: scaledFeatures

In [19]:
%%writefile largescalefinal1.py
from datetime import datetime
# Start time of training the model 

start = datetime.now()

from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, VectorIndexer, OneHotEncoder, Word2Vec, FeatureHasher 
from pyspark.ml.feature import StandardScaler, VectorAssembler 
from pyspark.ml.clustering import KMeans 
from pyspark.ml import Pipeline
from pyspark.sql.functions import split, regexp_replace
from pyspark.sql.types import StringType, IntegerType
from pyspark.sql.functions import col, regexp_replace, split, udf
from pyspark.ml.classification import DecisionTreeClassifier, RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator 
#import copy

# Initialize SparkSession
spark = SparkSession \
    .builder \
    .appName("Large Scale Project ") \
    .getOrCreate()


#Input files 
inputFileHire ="/user/user17/mosedata_proj/input/client_hiring_dt.csv"
inputFileBio ="/user/user17/mosedata_proj/input/client_bio_dt.csv"
inputFileCom = "/user/user17/mosedata_proj/input/client_communication_dt.csv"
inputFileAct = "/user/user17/mosedata_proj/input/client_activities_dt.csv"
inputFileFact = "/user/user17/mosedata_proj/input/client_fact_ft.csv"


# Create DataFrame from CSV file
dfHire =  spark.read.format("csv").option("sep", ",").option("inferSchema", "true").option("header", "true").load(inputFileHire)
dfBio =  spark.read.format("csv").option("sep", ",").option("inferSchema", "true").option("header", "true").load(inputFileBio)
dfCom =  spark.read.format("csv").option("sep", ",").option("inferSchema", "true").option("header", "true").load(inputFileCom)
dfAct =  spark.read.format("csv").option("sep", ",").option("inferSchema", "true").option("header", "true").load(inputFileAct)
dfFact =  spark.read.format("csv").option("sep", ",").option("inferSchema", "true").option("header", "true").load(inputFileFact)


# Print the schema of the DataFrames
dfHire.printSchema() 
dfBio.printSchema()
dfCom.printSchema()
dfAct.printSchema()
dfFact.printSchema()


#Temporary objects from which sql statements can be run for each dataframe
dfHire.createOrReplaceTempView("dfHire_sql")
dfBio.createOrReplaceTempView("dfBio_sql")
dfCom.createOrReplaceTempView("dfCom_sql")
dfAct.createOrReplaceTempView("dfAct_sql")
dfFact.createOrReplaceTempView("dfFAct_sql")

#Hired by Service branch
spark.sql(""" Select count(hires.id) as hired,bio.service_branch__c 
              From dfHire_sql hires 
              inner join dfBio_sql bio on bio.id = hires.id
              Where hires.hired = 1
              group by bio.service_branch__c
              order by hired asc
              """).show()

#Not hired by Service Branch
spark.sql(""" Select count(hires.id) as Not_hired,bio.service_branch__c 
              From dfHire_sql hires 
              inner join dfBio_sql bio on bio.id = hires.id
              Where hires.hired = 0
              group by bio.service_branch__c
              order by Not_hired asc
              """).show()

#Hired by rank
spark.sql(""" Select count(hires.id) as hired,bio.service_rank__c
              From dfHire_sql hires 
              inner join dfBio_sql bio on bio.id = hires.id
              Where hires.hired = 1
              group by bio.service_rank__c
              order by hired desc
              """).show()

#Prefered method of contact for the hired
spark.sql(""" select count(com.id) as count, com.preferred_method_of_contact__c 
              from dfCom_sql com 
              inner join dfHire_sql hires on com.id = hires.id
              where hires.hired = 1
              group by com.preferred_method_of_contact__c 
              order by count
          """).show()


#Prefered method of contact for the Unhired
spark.sql(""" select count(com.id) as count, com.preferred_method_of_contact__c 
              from dfCom_sql com 
              inner join dfHire_sql hires on com.id = hires.id
              where hires.hired = 0
              group by com.preferred_method_of_contact__c 
              order by count
          """).show()

#Select variables that determine whether one is going to be employed or not, store those varibles in a new dataframe
dfspark= spark.sql(""" select fact.yearsinservice as S_years , fact.reg_afterservice_years as R_years,
               act.resume_tailoring_tips__c as Resume_Tips,act.finalized_hhusa_revised_resume_on_file__c as Resume_OnFile, bio.client__c as Client, bio.client_type__c as Client_Type, bio.service_rank__c as Service_R
              ,bio.service_branch__c as Service_B, hire.hire_heroes_usa_confirmed_hire__c as HHUSA_hire, com.preferred_method_of_contact__c as Com_Method, com.responsive__c
              ,com.active__c, act.created_linkedin_account__c as Created_Linkedin, bio.highest_level_of_education_completed__c as Educ,  hire.hired
              ,bio.primary_military_occupational_specialty__c as Occupation
               
              from dfFact_sql fact
              
              inner join dfAct_sql act on act.id = fact.id
              inner join dfBio_sql bio on bio.id = fact.id
              inner join dfHire_sql hire on hire.id = fact.id
              inner join dfCom_sql com on com.id = fact.id
              
              where bio.client_type__c != '' and bio.service_branch__c !='' and bio.service_rank__c !='' 
              and com.preferred_method_of_contact__c !='' and fact.yearsinservice < 53 and fact.yearsinservice > 0 
              and fact.reg_afterservice_years >- 32 and fact.reg_afterservice_years < 53 and com.active__c !=''
              and bio.highest_level_of_education_completed__c !='' and hire.hired  !='' and hire.hired !='No' and hire.hired != 3

          """)
dfspark.persist()

dfsparkcopy = dfspark

# Get the number of clients in the description above
dfspark.count()

# Change the data type of the the hired column from string to double
dfspark = dfspark.withColumn("hired", dfspark["hired"].cast(DoubleType()))

# Confirm the labels in the hired column 0 represents the client was not hired 1 represents they were hired. 
dfspark.select('hired').distinct().show()


# The data types of the data frame 
print(" ")
print("The data types of the data frame. We need to convert them to indexes \
      vectors and then assemble into features, scale then apply to Machine learning algorithms. They accept only vectors" )
print(" ") 
print (dfspark.dtypes)
print(" ")
#Transform the data frame categorical columns to indexes using the StringIndexer method. 
indexers = [StringIndexer(inputCol=i, outputCol=i+"Index").fit(dfspark) \
            for i in list(set(dfspark.columns)- set(['S_years','R_years','Resume_Tips','Resume_OnFile','Client','HHUSA_hire','responsive__c','Created_Linkedin','Occupation','Educ'])) ]
pipeline = Pipeline(stages=indexers)
dfspark = pipeline.fit(dfspark).transform(dfspark)


#One hot encode the converted columns
encoder = OneHotEncoder(inputCols=["Service_BIndex", "Com_MethodIndex","active__cIndex", "Service_RIndex", "Client_TypeIndex"],
                       outputCols=["Service_BIndexVec", "Com_MethodIndexVec","active__cIndexVec", "Service_RIndexVec", "Client_TypeIndexVec"])
modelOne = encoder.fit(dfspark)
dfspark = modelOne.transform(dfspark)



#Convert the Education column from string to array of words using regular expressions. 
dfspark = dfspark.withColumn("Educ",split(regexp_replace("Educ", r"(^\[\[\[)|(\]\]\]$)", ""), ", ")
)

#Convert the words to vector in the Education column
word2Vec = Word2Vec(inputCol="Educ", outputCol="EducVec")
modelw = word2Vec.fit(dfspark)
dfspark = modelw.transform(dfspark)

# We can now put our newly created feature vectors  into one feature vector using the vectorizer. 
assembler2 = VectorAssembler(inputCols=['Resume_Tips','Resume_OnFile','Client','HHUSA_hire','responsive__c','Created_Linkedin','active__cIndexVec','Service_RIndexVec','Client_TypeIndexVec','Com_MethodIndexVec','Service_BIndexVec','EducVec'],outputCol="vecfeatures")
dfspark = assembler2.transform(dfspark)


# Scale the data uniformly
scaler2 = StandardScaler(inputCol="vecfeatures", outputCol="features")
scalerModel = scaler2.fit(dfspark)
dfspark = scalerModel.transform(dfspark)

#Train test split
train_data,test_data=dfspark.select('features','hired').randomSplit([0.7,0.3])

print("Classification with Decision trees and Random Forests. ")
#Decision tree classifier will predict whether a client can be hired or not 1 or 0 using the hired and scaledFeatures colums
decision_tree = DecisionTreeClassifier(labelCol='hired', featuresCol='features')
model1 = decision_tree.fit(train_data)
pred = model1.transform(test_data)

#Check the new columns of the dataframe 
pred.columns


# Now lets evaluate the algorithm performance
def evaluate_algo(df,label,Algorithm_name  ):
    classification_evaluator = MulticlassClassificationEvaluator(
    labelCol=label, predictionCol="prediction")
    accuracy = classification_evaluator.evaluate(df)
    print(Algorithm_name )
    print("Test Error  = %g " % (1.0 - accuracy))
    print("Accuracy   = %g " % accuracy)


#Evaluate the algorithm performance 
evaluate_algo(pred,'hired','Decision Tree')

#Compare with another machine learning algorithm Random forests 
print(" ")
random_forest = RandomForestClassifier(labelCol='hired', featuresCol='features', numTrees=28)
model2= random_forest.fit(train_data)
predrand = model2.transform(test_data)

#Columns predicted for random forests
predrand.columns

#Evaluate the performance of random forests
evaluate_algo(predrand,'hired','Random Forests')


#Clustering
print(" ")
print("Clustering with KMEANS")
funchash = FeatureHasher(inputCols= ['S_years','R_years','Resume_Tips','Resume_OnFile','Client','Client_Type','Service_R',\
 'Service_B','HHUSA_hire','Com_Method','responsive__c','active__c','Created_Linkedin','Educ','Occupation'],
                       outputCol="features")

dfsparkcopy = funchash.transform(dfsparkcopy)

def cluster(k,df,features):
    """ Takes in k as an int, the dataframe, and features """
    kmeans = KMeans(k=k, seed=1)  # 2 clusters here
    model = kmeans.fit(df.select(features))
    return model

transformeddf = cluster(4,dfsparkcopy,'features').transform(dfsparkcopy)

#Number of clusters

print ("Number of clusters from Kmeans")
transformeddf.select('prediction').distinct().show()

#Service branch with Service Rank, Showing whether hired or not. 
print(" ")
print(" Service branches with Service Ranks and cluster as prediction")
transformeddf.select('Service_B','Service_R','hired','prediction').distinct().show()


#Information about individual clusters in term of years service men stay in the army and when they register for services
print(" ")
print ("Statistical Information about years in service and Registration for services with HHUSA")
transformeddf.select('S_years','R_years').describe().show()

#Get cluster information Cluster 0 in terms of years service men spend on service.
print(" ")
print ("Years clustered with cluster 0 ")
cluster0 = transformeddf.filter(transformeddf.prediction==0)
cluster0.select('S_years','R_years').describe().show()

#Get cluster information Cluster 1

print ("Years clustered with cluster 1 ")
cluster1 = transformeddf.filter(transformeddf.prediction==1)
cluster1.select('S_years','R_years').describe().show()

#Get cluster information Cluster 2
print(" ")
print ("Years clustered with cluster 2")
cluster2 = transformeddf.filter(transformeddf.prediction==2)
cluster2.select('S_years','R_years').describe().show()


#Get cluster information Cluster 3
print(" ")
print ("Years clustered with cluster 3 ")
cluster3 = transformeddf.filter(transformeddf.prediction==3)
cluster3.select('S_years','R_years').describe().show()


end = datetime.now()

print('Duration of script: {}'.format(end - start))


Writing largescalefinal1.py


In [56]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator

# Loads data.
dataset = dfspark.select('features')

# Trains a k-means model.
kmeans = KMeans().setK(4).setSeed(1)
model = kmeans.fit(dataset)

# Make predictions
predictions = model.transform(dataset)

In [54]:
predictions.show()

+--------------------+----------+
|            features|prediction|
+--------------------+----------+
|[1.0,1.0,1.0,0.0,...|         3|
|[1.0,1.0,1.0,1.0,...|         1|
|[1.0,1.0,1.0,0.0,...|         0|
|[1.0,1.0,1.0,0.0,...|         2|
|[0.0,1.0,1.0,0.0,...|         3|
|[1.0,1.0,1.0,0.0,...|         3|
|[1.0,1.0,1.0,0.0,...|         0|
|[1.0,1.0,1.0,0.0,...|         3|
|[1.0,1.0,1.0,0.0,...|         3|
|[0.0,0.0,1.0,0.0,...|         3|
|[1.0,1.0,1.0,0.0,...|         0|
|[1.0,1.0,1.0,0.0,...|         0|
|[1.0,1.0,1.0,0.0,...|         3|
|[1.0,1.0,1.0,1.0,...|         0|
|[1.0,1.0,1.0,0.0,...|         0|
|[1.0,1.0,1.0,1.0,...|         0|
|[0.0,1.0,1.0,1.0,...|         1|
|[1.0,1.0,1.0,0.0,...|         3|
|[0.0,0.0,1.0,0.0,...|         2|
|[1.0,1.0,1.0,1.0,...|         3|
+--------------------+----------+
only showing top 20 rows



In [58]:
#Clustering 
def cluster(k,df,features):
    """ Takes in k as an int, the dataframe, and features """
    kmeans = KMeans(k=k, seed=1)  # 2 clusters here
    model = kmeans.fit(df.select(features))
    return model

transformeddf = cluster(4,dfspark,'features').transform(dfspark)

In [63]:
transformeddf.select('prediction').show()

+----------+
|prediction|
+----------+
|         3|
|         1|
|         0|
|         2|
|         3|
|         3|
|         0|
|         3|
|         3|
|         3|
|         0|
|         0|
|         3|
|         0|
|         0|
|         0|
|         1|
|         3|
|         2|
|         3|
+----------+
only showing top 20 rows

